# This is a test for extraction of radiomics features from the patients

In [1]:
import sys
import os
import json
import datetime
import subprocess

from params import get_folder


In [6]:
import radiomics
# import elikopy
import numpy as np
# import nibabel as nib
#import matplotlib.pyplot as plt
import SimpleITK as sitk
import logging
#import pandas as pd
import json

from scipy import stats
#from statsmodels.stats.weightstats import DescrStatsW
from radiomics import featureextractor, getFeatureClasses
#from elikopy.utils import submit_job

In [2]:
# Get the PyRadiomics logger (default log-level = INFO)
logger = radiomics.logger
logger.setLevel(logging.DEBUG)  # set level to DEBUG to include debug log messages in log file

# Write out all log entries to a file
handler = logging.FileHandler(filename='../testLog.txt', mode='w')
formatter = logging.Formatter('%(levelname)s:%(name)s: %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

In [7]:
folder_path = "../study"

Parameters for radiomcis feature extracion

In [8]:
params = folder_path + "/static_files/radiomics_params.yaml"

## Print some slices of patient 2

In [15]:
subj = "VNSLC_04"
subj_path = "%s/subjects/%s" % (folder_path, subj)
subj_path_freesurf = "%s/freesurfer/%s" % (folder_path, subj)

# image_path = "%s/registration/%s_T1_brain_reg.nii.gz" % (subj_path, subj)
# mask_path = "%s/registration/aparc+aseg_reg.mgz" % (subj_path)

image_path = "%s/dMRI/microstructure/dti/%s_FA.nii.gz" % (subj_path, subj)
mask_path = "%s/dlabel/diff/aparc+aseg+thalnuc.bbr.nii.gz" % (subj_path_freesurf)

image = sitk.ReadImage(image_path)
mask = sitk.ReadImage(mask_path)

# I didnt get this passage but it works
image = sitk.GetImageFromArray(sitk.GetArrayFromImage(image))
mask = sitk.GetImageFromArray(np.ones(sitk.GetArrayFromImage(image).shape))

In [74]:
# bb, correctedMask = imageoperations.checkMask(image, mask, label=10)
# if correctedMask is not None:
#     mask = correctedMask
# croppedImage, croppedMask = imageoperations.cropToTumorMask(image, mask, bb)
# 
# plt.imshow(sitk.GetArrayFromImage(croppedImage)[10, :, :], cmap="gray")
# plt.imshow(np.where(sitk.GetArrayFromImage(croppedMask)[10, :, :] == 10, 1, 0), alpha=0.5)
# plt.show()

In [75]:
print(image[60, 60, 30])

0.8405095338821411


## Load on pyRadiomics


In [11]:
extractor = featureextractor.RadiomicsFeatureExtractor(params)

extractor.enableAllImageTypes()
extractor.enableAllFeatures()

# ----- Test the mean, standard deviaton, skewness and kurtosis
# extractor.disableAllFeatures()
# extractor.enableFeaturesByName(firstorder=['Mean', 'StandardDeviation', 'Skewness', 'Kurtosis'])

print('Extraction parameters:\n\t', extractor.settings)
print('Enabled filters:\n\t', extractor.enabledImagetypes)
print('Enabled features:\n\t', extractor.enabledFeatures)

Extraction parameters:
	 {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkNearestNeighbor', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'weightingNorm': 'euclidean'}
Enabled filters:
	 {'Original': {}, 'Exponential': {}, 'Gradient': {}, 'LBP2D': {}, 'LBP3D': {}, 'LoG': {}, 'Logarithm': {}, 'Square': {}, 'SquareRoot': {}, 'Wavelet': {}}
Enabled features:
	 {'firstorder': [], 'shape': [], 'glcm': [], 'glszm': [], 'glrlm': [], 'ngtdm': [], 'gldm': [], 'shape2D': []}


In [9]:
featureClasses = getFeatureClasses()
print('Active features:')
for cls, features in extractor.enabledFeatures.items():
    if len(features) == 0:
        features = [f for f, deprecated in featureClasses[cls].getFeatureNames().items() if not deprecated]
    for f in features:
        print(f)
        print(getattr(featureClasses[cls], 'get%sFeatureValue' % f).__doc__)

Active features:
Mean

    **8. Mean**

    .. math::
      \textit{mean} = \frac{1}{N_p}\displaystyle\sum^{N_p}_{i=1}{\textbf{X}(i)}

    The average gray level intensity within the ROI.
    
StandardDeviation

    **15. Standard Deviation**

    .. math::
      \textit{standard deviation} = \sqrt{\frac{1}{N_p}\sum^{N_p}_{i=1}{(\textbf{X}(i)-\bar{X})^2}}

    Standard Deviation measures the amount of variation or dispersion from the Mean Value. By definition,
    :math:`\textit{standard deviation} = \sqrt{\textit{variance}}`

    .. note::
      As this feature is correlated with variance, it is marked so it is not enabled by default.
      To include this feature in the extraction, specify it by name in the enabled features
      (i.e. this feature will not be enabled if no individual features are specified (enabling 'all' features),
      but will be enabled when individual features are specified, including this feature).
      Not present in IBSI feature definitions (correlated wit

In [17]:
result = extractor.execute(image, mask, voxelBased=True)

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


In [20]:
result

OrderedDict([('diagnostics_Versions_PyRadiomics', '3.1.0a2'),
             ('diagnostics_Versions_Numpy', '1.25.2'),
             ('diagnostics_Versions_SimpleITK', '2.2.1'),
             ('diagnostics_Versions_PyWavelet', '1.4.1'),
             ('diagnostics_Versions_Python', '3.9.18'),
             ('diagnostics_Configuration_Settings',
              {'minimumROIDimensions': 2,
               'minimumROISize': None,
               'normalize': False,
               'normalizeScale': 1,
               'removeOutliers': None,
               'resampledPixelSpacing': None,
               'interpolator': 'sitkBSpline',
               'preCrop': False,
               'padDistance': 5,
               'distances': [1],
               'force2D': False,
               'force2Ddimension': 0,
               'resegmentRange': None,
               'label': 13,
               'additionalInfo': True}),
             ('diagnostics_Configuration_EnabledImageTypes', {'LoG': {}}),
             ('diagnost

In [ ]:
sitk.WriteImage(result, "test.nii.gz")

## Comparison with the metrics compute with nibabel, numpy and scipy

In [12]:
image = nib.load(image_path).get_fdata()
mask = nib.load(mask_path).get_fdata()

mask = np.where(mask == 13, 1, 0) # Take only the thalamus

In [13]:
assert image.shape == mask.shape

v = image.ravel()
w = mask.ravel()

assert v.size == w.size

# Mean
print("Mean", np.average(v, weights=w))
# Standard Deviation
dstat = DescrStatsW(v, w)
print("Std", dstat.std)
# Skewness
w_discrete = np.round(w).astype(int)
repeat = np.repeat(v, w_discrete)
print("Skewness", stats.skew(repeat, bias=True))
# Kurtosis
print("Kurtosis", stats.kurtosis(repeat, fisher=False, bias=True))


Mean 0.5063262347925882
Std 0.2388446199979036
Skewness 0.09805681068698882
Kurtosis 1.6332679206647815
